In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [4]:
agdb = pd.read_csv('psd_grains_pulses.csv')
agdb

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,430000,Barley,AF,Afghanistan,1960,2006,7,4,Area Harvested,4,(1000 HA),350.00
1,430000,Barley,AF,Afghanistan,1960,2006,7,20,Beginning Stocks,8,(1000 MT),0.00
2,430000,Barley,AF,Afghanistan,1960,2006,7,125,Domestic Consumption,8,(1000 MT),378.00
3,430000,Barley,AF,Afghanistan,1960,2006,7,176,Ending Stocks,8,(1000 MT),0.00
4,430000,Barley,AF,Afghanistan,1960,2006,7,88,Exports,8,(1000 MT),0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
535315,410000,Wheat,RH,Zimbabwe,2021,2022,2,86,Total Supply,8,(1000 MT),490.00
535316,410000,Wheat,RH,Zimbabwe,2021,2022,2,113,TY Exports,8,(1000 MT),0.00
535317,410000,Wheat,RH,Zimbabwe,2021,2022,2,84,TY Imp. from U.S.,8,(1000 MT),0.00
535318,410000,Wheat,RH,Zimbabwe,2021,2022,2,81,TY Imports,8,(1000 MT),150.00


In [5]:
agdb.dtypes

Commodity_Code             int64
Commodity_Description     object
Country_Code              object
Country_Name              object
Market_Year                int64
Calendar_Year              int64
Month                      int64
Attribute_ID               int64
Attribute_Description     object
Unit_ID                    int64
Unit_Description          object
Value                    float64
dtype: object

In [6]:
agdb.describe()

,Commodity_Code,Market_Year,Calendar_Year,Month,Attribute_ID,Unit_ID,Value
count,535320.000000,535320.000000,535320.000000,535320.000000,535320.000000,535320.000000,535320.000000
mean,434392.088097,1991.827841,2009.825572,6.682414,101.935037,8.933333,1510.250292
std,17610.593990,17.560671,5.538972,2.078532,59.172351,4.668575,10129.541300
min,410000.000000,1960.000000,2006.000000,1.000000,4.000000,4.000000,0.000000
25%,422110.000000,1977.000000,2006.000000,7.000000,57.000000,8.000000,0.000000
50%,430000.000000,1992.000000,2006.000000,7.000000,88.000000,8.000000,26.000000
75%,452000.000000,2007.000000,2014.000000,7.000000,176.000000,8.000000,310.000000
max,459900.000000,2021.000000,2022.000000,12.000000,192.000000,26.000000,504256.000000


In [7]:
agdb = agdb.dropna(how='any',axis=0)
agdb.shape

(535320, 12)

agdb['Market_Year'] = agdb.Market_Year
print(agdb['Market_Year'])
agdb

In [8]:
## Reducing columns that can't be used with the other databases

agdb_reduced = agdb.drop(['Calendar_Year', 'Country_Code', 'Month'], axis = 1)

In [9]:
agdb_reduced

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,430000,Barley,Afghanistan,1960,4,Area Harvested,4,(1000 HA),350.00
1,430000,Barley,Afghanistan,1960,20,Beginning Stocks,8,(1000 MT),0.00
2,430000,Barley,Afghanistan,1960,125,Domestic Consumption,8,(1000 MT),378.00
3,430000,Barley,Afghanistan,1960,176,Ending Stocks,8,(1000 MT),0.00
4,430000,Barley,Afghanistan,1960,88,Exports,8,(1000 MT),0.00
...,...,...,...,...,...,...,...,...,...
535315,410000,Wheat,Zimbabwe,2021,86,Total Supply,8,(1000 MT),490.00
535316,410000,Wheat,Zimbabwe,2021,113,TY Exports,8,(1000 MT),0.00
535317,410000,Wheat,Zimbabwe,2021,84,TY Imp. from U.S.,8,(1000 MT),0.00
535318,410000,Wheat,Zimbabwe,2021,81,TY Imports,8,(1000 MT),150.00


In [10]:
## Filtering the years to match the timespans of other databases

filter_years = []
for i in range(53):
  filter_years.append(1960 + i)
print(filter_years)

[1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]


In [11]:
agdb_timespan = agdb_reduced[(agdb_reduced['Market_Year'].isin(filter_years))]

In [12]:
agdb_timespan

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,430000,Barley,Afghanistan,1960,4,Area Harvested,4,(1000 HA),350.0
1,430000,Barley,Afghanistan,1960,20,Beginning Stocks,8,(1000 MT),0.0
2,430000,Barley,Afghanistan,1960,125,Domestic Consumption,8,(1000 MT),378.0
3,430000,Barley,Afghanistan,1960,176,Ending Stocks,8,(1000 MT),0.0
4,430000,Barley,Afghanistan,1960,88,Exports,8,(1000 MT),0.0
...,...,...,...,...,...,...,...,...,...
535180,410000,Wheat,Zimbabwe,2012,86,Total Supply,8,(1000 MT),336.0
535181,410000,Wheat,Zimbabwe,2012,113,TY Exports,8,(1000 MT),0.0
535182,410000,Wheat,Zimbabwe,2012,84,TY Imp. from U.S.,8,(1000 MT),0.0
535183,410000,Wheat,Zimbabwe,2012,81,TY Imports,8,(1000 MT),250.0


In [13]:
agdb_timespan.describe()

,Commodity_Code,Market_Year,Attribute_ID,Unit_ID,Value
count,452160.000000,452160.000000,452160.000000,452160.000000,452160.000000
mean,434371.645104,1987.194500,101.926048,8.933333,1387.089350
std,17601.419498,15.022572,59.172192,4.668576,8860.609499
min,410000.000000,1960.000000,4.000000,4.000000,0.000000
25%,422110.000000,1975.000000,57.000000,8.000000,0.000000
50%,430000.000000,1988.000000,88.000000,8.000000,25.000000
75%,452000.000000,2000.000000,176.000000,8.000000,285.000000
max,459900.000000,2012.000000,192.000000,26.000000,374637.000000


In [14]:
agdb_timespan.groupby(['Market_Year','Country_Name']).mean()

Commodity_Code  Attribute_ID   Unit_ID        Value
Market_Year Country_Name                                                     
1960        Afghanistan    425527.500000    101.633333  8.933333   467.566833
            Albania        436600.000000    103.066667  8.933333    39.710400
            Algeria        430822.000000    101.920000  8.933333   324.221333
            Angola         430403.333333    101.155556  8.933333   160.933556
            Argentina      440426.250000    102.350000  8.933333   725.071167
...                                  ...           ...       ...          ...
2012        Venezuela      432827.500000    101.633333  8.933333   787.537833
            Vietnam        425527.500000    101.633333  8.933333  3867.865000
            Yemen          436735.000000    102.111111  8.933333   355.088889
            Zambia         438082.000000    101.920000  8.933333   378.775333
            Zimbabwe       439660.000000    103.066667  8.933333   176.803600

[7101 rows x 4 columns]

In [15]:
## Using UniqueCountries.ipnyb to ensure the 

original_countries = agdb_timespan['Country_Name']

In [16]:
print(original_countries.unique())

['Afghanistan' 'Albania' 'Algeria' 'Argentina' 'Armenia' 'Australia'
 'Azerbaijan' 'Bangladesh' 'Belarus' 'Bhutan' 'Bolivia'
 'Bosnia and Herzegovina' 'Brazil' 'Bulgaria' 'Canada' 'Chile' 'China'
 'Colombia' 'Croatia' 'Cuba' 'Cyprus' 'Czech Republic' 'Ecuador' 'Egypt'
 'Eritrea' 'Estonia' 'Ethiopia' 'EU-15' 'European Union'
 'Former Czechoslovakia' 'Former Yugoslavia' 'Georgia' 'Hungary' 'Iceland'
 'India' 'Iran' 'Iraq' 'Israel' 'Japan' 'Jordan' 'Kazakhstan' 'Kenya'
 'Korea, South' 'Kuwait' 'Kyrgyzstan' 'Latvia' 'Lebanon' 'Libya'
 'Lithuania' 'Malta' 'Mexico' 'Moldova' 'Morocco' 'Nepal' 'New Zealand'
 'North Macedonia' 'Norway' 'Pakistan' 'Peru' 'Poland' 'Romania' 'Russia'
 'Saudi Arabia' 'Serbia' 'Serbia and Montenegro' 'Singapore' 'Slovakia'
 'Slovenia' 'South Africa' 'Switzerland' 'Syria' 'Taiwan' 'Tajikistan'
 'Thailand' 'Tunisia' 'Turkey' 'Turkmenistan' 'Ukraine'
 'Union of Soviet Socialist Repu' 'United Arab Emirates' 'United States'
 'Uruguay' 'Uzbekistan' 'Venezuela' 'Vietnam' 

In [17]:
len(original_countries.unique())

162

In [18]:
reduced_countries = ['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Benin', 'Bhutan', 'Bolivia', 'Botswana',
       'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia', 'Fiji', 'Gabon',
       'Georgia', 'Ghana', 'Guatemala', 'Guinea', 'Guyana', 'Haiti',
       'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Israel', 'Jamaica', 'Japan',
       'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait', 'Kyrgyzstan', 'Laos',
       'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania',
       'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Malta', 'Mauritania',
       'Mauritius', 'Mexico', 'Moldova', 'Mongolia', 'Morocco',
       'Mozambique', 'Namibia', 'Nepal', 'New Zealand', 'Nicaragua',
       'Niger', 'Nigeria', 'Norway', 'Oman', 'Pakistan', 'Panama',
       'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland',
       'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saudi Arabia', 'Senegal',
       'Serbia', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia',
       'Somalia', 'South Africa', 'Sri Lanka', 'Sudan', 'Suriname',
       'Switzerland', 'Syria', 'Taiwan', 'Tajikistan', 'Tanzania',
       'Thailand', 'Togo', 'Tunisia', 'Turkey', 'Turkmenistan', 'Uganda',
       'Ukraine', 'United Arab Emirates', 'United Kingdom',
       'United States', 'Uruguay', 'Uzbekistan', 'Venezuela', 'Vietnam',
       'Yemen', 'Zambia', 'Zimbabwe'
]

In [19]:
len(reduced_countries)

136

In [20]:
reduced_countries_agdb = agdb_timespan[(agdb_timespan['Country_Name'].isin(reduced_countries))]

In [21]:
## First major reduction of database, eliminated 129200 rows from countries not shared in all databases

reduced_countries_agdb

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,430000,Barley,Afghanistan,1960,4,Area Harvested,4,(1000 HA),350.0
1,430000,Barley,Afghanistan,1960,20,Beginning Stocks,8,(1000 MT),0.0
2,430000,Barley,Afghanistan,1960,125,Domestic Consumption,8,(1000 MT),378.0
3,430000,Barley,Afghanistan,1960,176,Ending Stocks,8,(1000 MT),0.0
4,430000,Barley,Afghanistan,1960,88,Exports,8,(1000 MT),0.0
...,...,...,...,...,...,...,...,...,...
535180,410000,Wheat,Zimbabwe,2012,86,Total Supply,8,(1000 MT),336.0
535181,410000,Wheat,Zimbabwe,2012,113,TY Exports,8,(1000 MT),0.0
535182,410000,Wheat,Zimbabwe,2012,84,TY Imp. from U.S.,8,(1000 MT),0.0
535183,410000,Wheat,Zimbabwe,2012,81,TY Imports,8,(1000 MT),250.0


In [23]:
reduced_countries_agdb.to_csv('callback_grains.csv', index=False)

In [ ]:
## Grouping the Import, Export and Yield values into a new column to further reduce the dataset
needed_att = ['Imports', 'Exports', 'Yield']

In [ ]:
reduced_countries_attributes_agdb = reduced_countries_agdb[(reduced_countries_agdb['Attribute_Description'].isin(needed_att))]
reduced_countries_attributes_agdb = reduced_countries_attributes_agdb.reset_index(drop=True)

In [ ]:
reduced_countries_attributes_agdb

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,430000,Barley,Afghanistan,1960,88,Exports,8,(1000 MT),0.00
1,430000,Barley,Afghanistan,1960,57,Imports,8,(1000 MT),0.00
2,430000,Barley,Afghanistan,1960,184,Yield,26,(MT/HA),1.08
3,430000,Barley,Afghanistan,1961,88,Exports,8,(1000 MT),0.00
4,430000,Barley,Afghanistan,1961,57,Imports,8,(1000 MT),0.00
...,...,...,...,...,...,...,...,...,...
79219,410000,Wheat,Zimbabwe,2011,57,Imports,8,(1000 MT),270.00
79220,410000,Wheat,Zimbabwe,2011,184,Yield,26,(MT/HA),3.31
79221,410000,Wheat,Zimbabwe,2012,88,Exports,8,(1000 MT),0.00
79222,410000,Wheat,Zimbabwe,2012,57,Imports,8,(1000 MT),250.00


Deriving attributes from values of filtered dataset

In [ ]:
imports = reduced_countries_attributes_agdb[reduced_countries_attributes_agdb['Attribute_Description'] == 'Imports']['Value']
imports

1          0.0
4          0.0
7          0.0
10         0.0
13         0.0
         ...  
79210    200.0
79213    250.0
79216    180.0
79219    270.0
79222    250.0
Name: Value, Length: 26408, dtype: float64

In [ ]:
reduced_countries_attributes_agdb['Import_Values(MT)']=imports * 1000

In [ ]:
exports = reduced_countries_attributes_agdb[reduced_countries_attributes_agdb['Attribute_Description'] == 'Exports']['Value']
exports

0        0.0
3        0.0
6        0.0
9        0.0
12       0.0
        ... 
79209    0.0
79212    0.0
79215    0.0
79218    0.0
79221    0.0
Name: Value, Length: 26408, dtype: float64

In [ ]:
reduced_countries_attributes_agdb['Export_Values(MT)']=exports * 1000

In [ ]:
reduced_countries_attributes_agdb

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value,Import_Values(MT),Export_Values(MT)
0,430000,Barley,Afghanistan,1960,88,Exports,8,(1000 MT),0.00,NaN,0.0
1,430000,Barley,Afghanistan,1960,57,Imports,8,(1000 MT),0.00,0.0,NaN
2,430000,Barley,Afghanistan,1960,184,Yield,26,(MT/HA),1.08,NaN,NaN
3,430000,Barley,Afghanistan,1961,88,Exports,8,(1000 MT),0.00,NaN,0.0
4,430000,Barley,Afghanistan,1961,57,Imports,8,(1000 MT),0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
79219,410000,Wheat,Zimbabwe,2011,57,Imports,8,(1000 MT),270.00,270000.0,NaN
79220,410000,Wheat,Zimbabwe,2011,184,Yield,26,(MT/HA),3.31,NaN,NaN
79221,410000,Wheat,Zimbabwe,2012,88,Exports,8,(1000 MT),0.00,NaN,0.0
79222,410000,Wheat,Zimbabwe,2012,57,Imports,8,(1000 MT),250.00,250000.0,NaN


In [ ]:
yields = reduced_countries_attributes_agdb[reduced_countries_attributes_agdb['Attribute_Description'] == 'Yield']['Value']
yields

2        1.08
5        1.08
8        1.08
11       1.08
14       1.09
         ... 
79211    1.09
79214    2.29
79217    2.93
79220    3.31
79223    3.40
Name: Value, Length: 26408, dtype: float64

In [ ]:
reduced_countries_attributes_agdb['Yield_Values(MT/HA)']=yields

In [ ]:
reduced_countries_attributes_agdb

In [ ]:
reduced_countries_attributes_agdb['Export_Values(MT)']

0        0.0
1        NaN
2        NaN
3        0.0
4        NaN
        ... 
79219    NaN
79220    NaN
79221    0.0
79222    NaN
79223    NaN
Name: Export_Values(MT), Length: 79224, dtype: float64

In [ ]:
reduced_countries_attributes_agdb.to_csv('PSD_Grains_Edit1.csv', index=False)

In [ ]:
df = pd.read_csv('PSD_Grains_Edit1.csv')

In [ ]:
df

,Commodity_Code,Commodity_Description,Country_Name,Market_Year,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value,Import_Values(MT),Export_Values(MT),Yield_Values(MT/HA)
0,430000,Barley,Afghanistan,1960,88,Exports,8,(1000 MT),0.00,NaN,0.0,NaN
1,430000,Barley,Afghanistan,1960,57,Imports,8,(1000 MT),0.00,0.0,NaN,NaN
2,430000,Barley,Afghanistan,1960,184,Yield,26,(MT/HA),1.08,NaN,NaN,1.08
3,430000,Barley,Afghanistan,1961,88,Exports,8,(1000 MT),0.00,NaN,0.0,NaN
4,430000,Barley,Afghanistan,1961,57,Imports,8,(1000 MT),0.00,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79219,410000,Wheat,Zimbabwe,2011,57,Imports,8,(1000 MT),270.00,270000.0,NaN,NaN
79220,410000,Wheat,Zimbabwe,2011,184,Yield,26,(MT/HA),3.31,NaN,NaN,3.31
79221,410000,Wheat,Zimbabwe,2012,88,Exports,8,(1000 MT),0.00,NaN,0.0,NaN
79222,410000,Wheat,Zimbabwe,2012,57,Imports,8,(1000 MT),250.00,250000.0,NaN,NaN


In [ ]:
mod_df = df.groupby('Country_Name')

In [ ]:
print(mod_df)

In [ ]:

#agdb_imports = reduced_countries_agdb.loc[(reduced_countries_agdb['Attribute_Description'] == 'Imports')]

In [ ]:
#agdb_imports

In [ ]:
## Creating an updated dataset for sharing

#reduced_countries_agdb.to_csv('PSD_Grains_Pulses_Reduced_Countries_Edit.csv')